# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = "../WeatherPy/output_data/cities.csv"
df = pd.read_csv(path)
del df["City_ID"]
df.head()

,City,Lat,Lng,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,rikitea,-23.12,-134.97,69.57,68,0,21.92,PF,1593656666
1,cape town,-33.93,18.42,53.01,100,33,3.36,ZA,1593656425
2,pekan nenas,1.51,103.51,84.20,79,20,4.70,MY,1593657237
3,east london,-33.02,27.91,63.52,32,52,5.10,ZA,1593656650
4,sile,41.18,29.61,72.00,82,0,8.30,TR,1593657239


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [8]:
locations = df[["Lat", "Lng"]]
humidity = df["Humidity (%)"]

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
df.head()

,City,Lat,Lng,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,rikitea,-23.12,-134.97,69.57,68,0,21.92,PF,1593656666
1,cape town,-33.93,18.42,53.01,100,33,3.36,ZA,1593656425
2,pekan nenas,1.51,103.51,84.20,79,20,4.70,MY,1593657237
3,east london,-33.02,27.91,63.52,32,52,5.10,ZA,1593656650
4,sile,41.18,29.61,72.00,82,0,8.30,TR,1593657239


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 570 non-null    object 
 1   Lat                  570 non-null    float64
 2   Lng                  570 non-null    float64
 3   Max Temperature (F)  570 non-null    float64
 4   Humidity (%)         570 non-null    int64  
 5   Cloudiness (%)       570 non-null    int64  
 6   Wind Speed (mph)     570 non-null    float64
 7   Country              565 non-null    object 
 8   Date                 570 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 40.2+ KB


In [31]:
hotel_df = df.loc[(df["Max Temperature (F)"] < 75) & (df["Max Temperature (F)"] > 70) & (df["Wind Speed (mph)"] < 7) & (df["Cloudiness (%)"] == 0), :]
hotel_df

,City,Lat,Lng,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
47,maceio,-9.67,-35.74,71.60,94,0,3.36,BR,1593657262
61,misratah,32.38,15.09,73.72,68,0,3.29,LY,1593657270
114,palafrugell,41.92,3.16,72.00,51,0,4.65,ES,1593657298
170,tabuleiro do norte,-5.25,-38.13,73.76,88,0,5.55,BR,1593657329
206,piacabucu,-10.41,-36.43,71.33,76,0,2.15,BR,1593657348
235,touros,-5.20,-35.46,71.60,100,0,3.36,BR,1593657363
275,hun,29.13,15.95,71.22,46,0,4.97,LY,1593657383
403,beyneu,45.32,55.20,71.58,31,0,4.21,KZ,1593657453
491,porto torres,40.84,8.40,71.60,88,0,1.12,IT,1593656918
544,lagunas,-5.23,-75.68,73.36,89,0,1.54,PE,1593657528


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df["Hotel Name"] = ""

In [35]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        row["Hotel Name"] = hotel_data["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [36]:
hotel_df

,City,Lat,Lng,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
47,maceio,-9.67,-35.74,71.60,94,0,3.36,BR,1593657262,Meridiano Hotel
61,misratah,32.38,15.09,73.72,68,0,3.29,LY,1593657270,Safari Hotel
114,palafrugell,41.92,3.16,72.00,51,0,4.65,ES,1593657298,Hotel La Malcontenta
170,tabuleiro do norte,-5.25,-38.13,73.76,88,0,5.55,BR,1593657329,Hotel Central II
206,piacabucu,-10.41,-36.43,71.33,76,0,2.15,BR,1593657348,Pousada Raizes
235,touros,-5.20,-35.46,71.60,100,0,3.36,BR,1593657363,Pousada Amigos do Vento
275,hun,29.13,15.95,71.22,46,0,4.97,LY,1593657383,Alrwasi Hotel
403,beyneu,45.32,55.20,71.58,31,0,4.21,KZ,1593657453,"HOTEL ""AKNUR"""
491,porto torres,40.84,8.40,71.60,88,0,1.12,IT,1593656918,Hotel Libyssonis
544,lagunas,-5.23,-75.68,73.36,89,0,1.54,PE,1593657528,Hospedaje Eco


In [ ]:
marker_locations = hotel_df[["Latitude", "Longitude"]]

fig = gmaps.figure()

markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
fig.add_layer(markers)
fig

In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))